In [6]:
from pepys_import.core.store.data_store import DataStore
import config
import pandas as pd
from sqlalchemy import create_engine
import geopandas as gpd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import hvplot
import hvplot.pandas
import folium

## Create DataStore object, and older-style SQLAlchemy object

In [2]:
ds = DataStore(
            db_username=config.DB_USERNAME,
            db_password=config.DB_PASSWORD,
            db_host=config.DB_HOST,
            db_port=config.DB_PORT,
            db_name=config.DB_NAME,
            db_type=config.DB_TYPE,
        )

   @@@@ @@@@@        ______                      _                            _   
 @@@@@ @@..@@@@      | ___ \                    (_)                          | |  
  @@@     @@@@@@     | |_/ /__ _ __  _   _ ___   _ _ __ ___  _ __   ___  _ __| |_ 
  @@@     @@@@@@@@   |  __/ _ \ '_ \| | | / __| | | '_ ` _ \| '_ \ / _ \| '__| __|
 @@@@.     @@@@@@@   | | |  __/ |_) | |_| \__ \ | | | | | | | |_) | (_) | |  | |_ 
 @@ #@@ @   ...@@@@@ \_|  \___| .__/ \__, |___/ |_|_| |_| |_| .__/ \___/|_|   \__|
 @@   &        @@@@@            | |     __/ | ______          | |                   
Software Version :  0.0.33



Database Type :  postgres
Database Name :  pepys_solent
Database Host :  localhost
------------------------------------------------------------------------------


In [3]:
engine = create_engine(ds.connection_string, future=False)

## Functions for plotting platforms and datafiles

In [ ]:
def plot_platform(platform_id):
    sql = f"SELECT * FROM pepys.states_for(null, null, null, null, null, '{{{platform_id}}}')"
    data = gpd.read_postgis(sql,
                            engine, geom_col='state_location')
    data['time_str'] = data.state_time.astype(str)
    data = data.dropna(subset=['state_location'])
    if len(data) == 0:
        return "No data"
    m = data[['time_str', 'state_location', 'heading', 'course', 'speed']].explore(column='speed')
    return m

In [ ]:
def plot_datafile(datafile_id):
    sql = f"SELECT * FROM pepys.states_for(null, null, null, null, '{{{datafile_id}}}', null)"
    data = gpd.read_postgis(sql,
                            engine, geom_col='state_location')
    data['time_str'] = data.state_time.astype(str)
    if len(data) == 0:
        return "No data"
    data = data.dropna(subset=['state_location'])
    m = data[['time_str', 'state_location', 'heading', 'course', 'speed']].explore(column='speed')
    return m

## Interactive UI to select platform and see plot

In [ ]:
with ds.session_scope():
    platforms = ds.session.query(ds.db_classes.Platform.name, ds.db_classes.Platform.platform_id).all()

In [ ]:
platforms = [(name, str(plat_id)) for name, plat_id in platforms]

In [ ]:
interact(plot_platform, platform_id=platforms)

## Interactive UI to select datafile and see plot

In [ ]:
with ds.session_scope():
    datafiles = ds.session.query(ds.db_classes.Datafile.reference, ds.db_classes.Datafile.datafile_id).all()

In [ ]:
datafiles = [(name, str(df_id)) for name, df_id in datafiles]

In [ ]:
interact(plot_datafile, datafile_id=datafiles)

## Data plots for the HIPP platform

#### Example static plot using matplotlib

In [ ]:
hipp_id = dict(platforms)['HIPP']

In [ ]:
sql = f"SELECT * FROM pepys.states_for(null, null, null, null, null, '{{{hipp_id}}}')"
data = gpd.read_postgis(sql,
                        engine, geom_col='state_location')

In [ ]:
_ = data.plot(kind='line', x='state_time', y='speed', grid=True, figsize=(10, 8))
plt.xlabel("Time")
plt.ylabel("Speed (m/s)")
plt.title("Example static plot")

#### Example interactive plot using hvplot

In [ ]:
pd.options.plotting.backend = 'hvplot'

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df = df.set_index(df.state_time)

In [ ]:
df = df.sort_index()

In [ ]:
df.plot(kind='line', y='speed', label="Raw speed", grid=True, title="Example interactive plot (speed vs time)", xlabel="Time", ylabel="Speed (m/s)") * df.speed.rolling(10).mean().plot(kind='line', label="Smoothed speed")

In [ ]:
len(df)

## Finding vessels within distance

In [65]:
def find_within_distance(selected_vessel_sensor_id, distance_threshold=500, time_threshold=1, just_map=False):
    sql = f"""SELECT location, sensor_id::text, time::text from pepys."States"
   WHERE pepys."States".sensor_id = '{selected_vessel_sensor_id}'"""

    selected_vessel = gpd.read_postgis(sql, engine, geom_col="location")
    
    if len(selected_vessel) == 0:
        return "No data"

    sql = f"""WITH selected_vessel AS
      (SELECT * from pepys."States"
       WHERE pepys."States".sensor_id = '{selected_vessel_sensor_id}'),
    non_selected_vessel AS
       (SELECT * from pepys."States"
       WHERE pepys."States".sensor_id != '{selected_vessel_sensor_id}')
    SELECT non_selected_vessel.time::text,
           non_selected_vessel.sensor_id::text,
           non_selected_vessel.location,
           pepys."Sensors".host::text,
           pepys."Platforms".name,
           ST_Distance(non_selected_vessel.location::geography, selected_vessel.location::geography) as dist
    FROM selected_vessel, non_selected_vessel
    LEFT JOIN pepys."Sensors" ON non_selected_vessel.sensor_id = pepys."Sensors".sensor_id
    LEFT JOIN pepys."Platforms" ON pepys."Sensors".host = pepys."Platforms".platform_id
    WHERE
       (selected_vessel.time - non_selected_vessel.time) > '-{time_threshold} minute'::interval
       and (selected_vessel.time - non_selected_vessel.time) < '{time_threshold} minute'::interval
       and ST_DWithin(selected_vessel.location::geography, non_selected_vessel.location::geography, {distance_threshold}); """

    close_vessels = gpd.read_postgis(sql, engine, geom_col="location")
    
#     m = folium.Map(control_scale=True)
    m = selected_vessel.explore(tiles='https://tiles.wmflabs.org/bw-mapnik/{z}/{x}/{y}.png', attr="OSM")
    close_vessels.explore(column='name', m=m, cmap='Set1')
    
    if just_map:
        return m

    return selected_vessel, close_vessels, m    

In [ ]:
selected_vessel, close_vessels, m = find_within_distance('5676bde2-1481-47f0-b259-f057009081d3', 1000, 1)

In [ ]:
m

In [16]:
with ds.session_scope():
    platforms = ds.session.query(ds.db_classes.Platform).all()

In [26]:
sensors = [(platform.name, str(platform.get_sensor(ds, 'AIS').sensor_id)) for platform in platforms]
sensors = sensors[1:]
sensors = sorted(sensors, key=lambda x: x[0])

In [27]:
interact(find_within_distance,
         selected_vessel_sensor_id=sensors,
         distance_threshold=(100, 1000, 100),
         time_threshold=fixed(1),
         just_map=fixed(True))

interactive(children=(Dropdown(description='selected_vessel_sensor_id', options=(('212068000', 'eaacf56a-e872-…

<function __main__.find_within_distance(selected_vessel_sensor_id, distance_threshold=500, time_threshold=1, just_map=False)>

In [83]:
%%time
selected_vessel, close_vessels, m = find_within_distance('5676bde2-1481-47f0-b259-f057009081d3', 1000, 1)

CPU times: user 164 ms, sys: 5.72 ms, total: 169 ms
Wall time: 482 ms


In [84]:
close_vessels = close_vessels.set_index(pd.DatetimeIndex(close_vessels.time))

In [85]:
vessels_in_range_by_time = close_vessels.resample('5T').apply(lambda x: len(x.groupby('name').groups))

In [92]:
pd.options.plotting.backend = 'holoviews'

In [104]:
vessels_in_range_by_time.plot(xlabel="Time", ylabel="Number of vessels within distance")

:Curve   [time]   (0)

In [103]:
close_vessels.dist.hist(xlabel="Distance (m)", ylabel="Frequency")

:Histogram   [dist]   (dist_count)

## Advanced SQL Playground

Old code below

In [ ]:
# Create index on conversion to Geography, so we can deal with distances in metres
CREATE INDEX ON pepys."States" USING GIST (CAST(location as geography));

In [ ]:
%%time
sql = """SELECT location from pepys."States" where
ST_DWithin(ST_GeomFromText('POINT(-80.91804208527637 29.509684234606357)', 4326), location, 0.4)
union all select ST_GeomFromText('POINT(-80.91804208527637 29.509684234606357)') as location;"""

df = gpd.read_postgis(sql, engine, geom_col="location")

In [ ]:
with ds.session_scope():
    states = ds.session.query(ds.db_classes.State).count()
    platforms = ds.session.query(ds.db_classes.Platform).count()
    
print(f"States: {states}")
print(f"Platforms: {platforms}")

In [ ]:
%%time
sql = """SELECT location from pepys."States" where
ST_DWithin(ST_GeomFromText('POINT(-80.91804208527637 29.509684234606357)', 4326)::geography, location::geography, 40000)
union all select ST_GeomFromText('POINT(-80.91804208527637 29.509684234606357)')::geography as location;"""

df = gpd.read_postgis(sql, engine, geom_col="location")

In [ ]:
%%time
sensor_id = '5676bde2-1481-47f0-b259-f057009081d3'
sql = f"""WITH selected_vessel AS
  (SELECT * from pepys."States"
   WHERE pepys."States".sensor_id = '{sensor_id}'),
non_selected_vessel AS
   (SELECT * from pepys."States"
   WHERE pepys."States".sensor_id != '{sensor_id}')
SELECT non_selected_vessel.time::text, non_selected_vessel.sensor_id::text, non_selected_vessel.location FROM selected_vessel, non_selected_vessel where (selected_vessel.time - non_selected_vessel.time) > '-1 minute'::interval and (selected_vessel.time - non_selected_vessel.time) < '1 minute'::interval and ST_DWithin(selected_vessel.location::geography, non_selected_vessel.location::geography, 500); """

df = gpd.read_postgis(sql, engine, geom_col="location")

In [ ]:
len(df)

In [ ]:
import folium

In [ ]:
m = folium.Map(control_scale=True)

In [ ]:
%%time
sql = f"""SELECT location, sensor_id::text, time::text from pepys."States"
   WHERE pepys."States".sensor_id = '{sensor_id}'"""

selected_vessel = gpd.read_postgis(sql, engine, geom_col="location")

In [ ]:
selected_vessel['sensor_id'] = selected_vessel['sensor_id'].astype(str)

In [ ]:
selected_vessel.explore(m=m)

In [ ]:
df['sensor_id'] = df['sensor_id'].astype(str)

In [ ]:
df.explore(m=m, color='red')

In [ ]:
df['col'] = '0'

In [ ]:
df.loc[df.index[-1], 'col'] = 1

In [ ]:
df[['location']]

In [ ]:
df[['location', 'col']].explore(column='col', cmap='viridis')

In [ ]:
plot_platform('dc2adf61-6b03-46db-8d78-303828ff2570')

In [ ]:
%%time
sensor_id = '5676bde2-1481-47f0-b259-f057009081d3'
sql = f"""WITH selected_vessel AS
  (SELECT * from pepys."States"
   WHERE pepys."States".sensor_id = '{sensor_id}'),
non_selected_vessel AS
   (SELECT * from pepys."States"
   WHERE pepys."States".sensor_id != '{sensor_id}')
SELECT ST_MakeLine(non_selected_vessel.location, selected_vessel.location) as line, non_selected_vessel.time::text, non_selected_vessel.sensor_id::text, non_selected_vessel.location FROM selected_vessel, non_selected_vessel where (selected_vessel.time - non_selected_vessel.time) > '-1 minute'::interval and (selected_vessel.time - non_selected_vessel.time) < '1 minute'::interval and ST_DWithin(selected_vessel.location::geography, non_selected_vessel.location::geography, 200); """

df = gpd.read_postgis(sql, engine, geom_col="line")

In [ ]:
df.explore(m=m, color='yellow')